In [1]:
#get the probabilities

In [ ]:
import pandas as pd
import os
import re
from nltk.stem.porter import *

In [ ]:
data = pd.read_csv("Review_Subset.csv")

In [ ]:
def getNgrams(text, n):
    if n == 1:
        return text
    
    text.append("<end>")
    ngramList = []
    for i in range(len(text) - (n-1)):
        ngram = text[i:i+n]
        ngramList.append(ngram)
    
    return ngramList

In [ ]:
unigramDict = {}
bigramDict = {}
stemmer = PorterStemmer()

def getCountsinRow(row):
    text = row["text"]
    parsedText = re.findall("(\w+)", text)
    
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        
    unigram = getNgrams(parsedText, 1)
    bigram = getNgrams(parsedText, 2)
    ngrams = (unigram,bigram)
    for index in range(len(ngrams)):
        for eachWord in ngrams[index]:
            
            if index == 0:
                #unigram
                
                if eachWord in unigramDict:
                    unigramDict[eachWord] +=1
                else:
                    unigramDict[eachWord] =1
                    
            if index == 1:
                #bigram
                    eachWord = tuple(eachWord)
                    if eachWord in bigramDict:
                        bigramDict[eachWord] +=1
                    else:
                        bigramDict[eachWord] =1

            

In [ ]:

#get the probabilies into unigramsDict and bigramsDict
data.apply(getCountsinRow, axis = 1)

In [ ]:
print(len(unigramDict))
print(len(bigramDict))

In [ ]:
#export unigrams
extractList = []
for eachKey in unigramDict:
    extractList.append([eachKey,unigramDict[eachKey]])

pd.DataFrame(extractList, columns = ["word", "count"]).to_csv("unigramProbabilities.csv", index = False)

In [ ]:
#export bigrams
extractList = []
for eachKey in bigramDict:
    extractList.append([eachKey[0],eachKey[1],bigramDict[eachKey]])
    
pd.DataFrame(extractList, columns = ["first", "second", "count"]).to_csv("bigramProbabilities.csv", index = False)

In [ ]:
print(data)

In [ ]:
#get the vector model in data
unigramVM = []
def getUnigramReviewVectors(row):
    
    #we have data,unigrams,bigrams
    rowVector = {}
    text = row["text"]
    score = row["score"]
    parsedText = re.findall("(\w+)", text)
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        #score = unigramDict[parsedText[index]]
        if parsedText[index] in rowVector:
            rowVector[parsedText] +=1
        else:
            rowVector[parsedText] = 1
    
    if score > 4:
        rowVector["<label>"] = "good"
    else:
        rowVector["<label>"] = "bad"
        
    unigramVM.append(rowVector)
    

data.apply(getUnigramReviewVectors, axis = 1)
    
    
    

In [ ]:
pd.DataFrame(unigramVM).to_csv("UnigramVectorModel.csv", index = False)

In [ ]:
len(unigramVM)
len(unigramDict)